This is the second notebook I'll be using for my LDA model with the customer complaints concerning the student loan data. I'm going to make a few adustments and hopefully get a better score than I did on my last model. I'm doing a different notebook to compare the model and process to the last one so nothing gets lost. The Coherence Score I got from the last model was 0.4441 (which was with 8 topics), which isn't all that good, so here I will see if I can get it above 0.6.

In [ ]:
!pip install nltk==3.5

     |████████████████████████████████| 1.4 MB 13.8 MB/s 
     |████████████████████████████████| 82 kB 2.1 MB/s 
     |████████████████████████████████| 666 kB 69.2 MB/s 
     |████████████████████████████████| 70 kB 21.6 MB/s 
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434674 sha256=3d7e4b6e695d2923d9831c337bb28697ef3ef694daf24eb53a313e03890adb73
  Stored in directory: /home/jovyan/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
# Make sure to have nltk and stopwords downloaded
import nltk; nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
!pip install gensim==3.8.3

     |████████████████████████████████| 24.2 MB 19.9 MB/s 
     |████████████████████████████████| 113 kB 82.2 MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-py3-none-any.whl size=107095 sha256=568fe9eb61bbbba91667a9a2ef3501c6d4631b6792f7b0d3ceff77661d64b4c8
  Stored in directory: /home/jovyan/.cache/pip/wheels/83/a6/12/bf3c1a667bde4251be5b7a3368b2d604c9af2105b5c1cb1870
Successfully built smart-open
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
!pip install spacy==2.3.2

     |████████████████████████████████| 9.9 MB 12.4 MB/s 
     |████████████████████████████████| 3.7 MB 73.5 MB/s 
     |████████████████████████████████| 185 kB 97.1 MB/s 
     |████████████████████████████████| 2.1 MB 66.3 MB/s 
     |████████████████████████████████| 118 kB 90.4 MB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
!pip install pyLDAvis==2.1.2

     |████████████████████████████████| 1.6 MB 18.8 MB/s 
     |████████████████████████████████| 162 kB 92.1 MB/s 
     |████████████████████████████████| 272 kB 90.7 MB/s 
     |████████████████████████████████| 829 kB 76.3 MB/s 
     |████████████████████████████████| 99 kB 17.4 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=02dcaf411ea6e23944d89a8a9a62d055d1b8d8c2c5387a79e74f56923a48c2fb
  Stored in directory: /home/jovyan/.cache/pip/wheels/3b/fb/41/e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491059 sha256=919c6c4df92b6beb24df703f79e7e4b8f98b183e3a7a359e34daa4643919d1d2
  Stored in directory: /home/jovyan/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built pyLDAvis future
  Attempting uninstall: pluggy
    Found existing installation: pluggy 1.0.0.dev0
    Uninstalling pluggy-1.0.0.dev0:
      Successfully u

In [ ]:
# Import needed packages

import re
import numpy
import pandas as pd
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

import logging
logging.basicConfig(format="%(asctime)s: %(levelname)s : %(message)s", level=logging.ERROR)

import warnings 
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
# NLTK stop words
from nltk.corpus import stopwords
stop_words = stopwords.words("english")
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
# Get data
df = pd.read_csv('../data/text_analysis_data.csv')
df.head()

,Date received,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company,State,Tags,Company response to consumer,Timely response?,Consumer disputed?,month,year
0,2020-05-19,Private student loan,Dealing with your lender or servicer,Received bad information about your loan,When I was applying for my loan my XXXX accoun...,"Figure Technologies, Inc",NJ,None,Closed with explanation,Yes,NaN,5,2020
1,2020-02-06,Federal student loan servicing,Incorrect information on your report,Account status incorrect,I'm on a deferred payment plan t never ; late,"Nelnet, Inc.",TX,None,Closed with explanation,Yes,NaN,2,2020
2,2020-02-08,Federal student loan servicing,Dealing with your lender or servicer,Problem with customer service,I have attempted multiple times to contact FED...,AES/PHEAA,KY,None,Closed with non-monetary relief,Yes,NaN,2,2020
3,2020-01-21,Federal student loan servicing,Dealing with your lender or servicer,Trouble with how payments are being handled,I was divorced in 2004 and I agreed to take th...,AES/PHEAA,OK,None,Closed with explanation,Yes,NaN,1,2020
4,2019-12-04,Federal student loan servicing,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,This particular account situation that is late...,AES/PHEAA,FL,None,Closed with explanation,Yes,NaN,12,2019


Now that we've imported the necessary packages we will prepare the data to build and feed into the model.

In [ ]:
!pip install textblob==0.15.3

     |████████████████████████████████| 636 kB 11.5 MB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
!python -m textblob.download_corpora
# A function to preprocess all rows in a dataframe
def preprocess_data(data):
    # Change all text to lowercase
    data = data.apply(lambda x: " ".join(x.lower() for x in x.split()))
    
    # Remove puctuation
    data = data.str.replace("[^\w\s]","")
    
    # Remove stopwords
    from nltk.corpus import stopwords
    stop = stopwords.words("english")
    data = data.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    
    # Remove any words like "xx" words and numbers
    data = data.apply(lambda x: " ".join(x for x in x.split() if "xx" not in x))
    data = data.apply(lambda x: " ".join(x for x in x.split() if not x.isnumeric()))
    # This round we decided not to remove common words
#     freq = pd.Series(" ".join(data).split()).value_counts()[:10]
#     freq = list(freq.index)
#     data = data.apply(lambda x: " ".join(x for x in x.split() if x not in freq))
    
    # Lemmatization
    from textblob import Word
    data = data.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
    
    # Return transformed data
    return data

# Return the data as a list
data = preprocess_data(df["Consumer complaint narrative"])

[nltk_data] Downloading package brown to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package conll2000 to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [ ]:
# Tokenize the data
data = [sub.split() for sub in data] 
print(data[:3])

[['applying', 'loan', 'account', 'correctly', 'communicate', 'issue', 'offer', 'rate', 'deduction', 'autopay', 'showing', 'account', 'told', 'go', 'application', 'anyway', 'account', 'opened', 'could', 'add', 'autopay', 'receive', 'discount', 'way', 'since', 'account', 'opened', 'called', 'call', 'center', 'least', 'time', 'trying', 'receive', 'autopay', 'discount', 'first', 'time', 'told', 'going', 'applied', 'still', 'seen', 'additionally', 'last', 'time', 'called', 'week', 'ago', 'asked', 'speak', 'manager', 'told', 'would', 'take', 'day', 'get', 'back', 'still', 'yet', 'hear', 'back', 'business', 'day', 'later', 'told', 'would', 'receiving', 'autopay', 'discount', 'receiving', 'opened', 'account', 'company', 'lying', 'rate', 'loan', 'going', 'receive', 'dont', 'autopay', 'initiate', 'autopays', 'go', 'far', 'additional', 'issue', 'told', 'rate', 'going', 'based', 'month', 'libor', 'rate', 'published', 'wsj', 'month', 'none', 'rate', 'received', 'thus', 'far', 'match', 'rate', 'dont

Now that we've cleaned and tokenized the data we need to create bigrams and trigrams. Bigrams are two words frequently seen paired together, trigrams are the same but with three words. We will use Gensim's Phrases model to build the bigrams and trigrams.

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data, min_count=5, threshold=20)
trigram = gensim.models.Phrases(bigram[data], threshold=20)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data[0]]])

['applying', 'loan', 'account', 'correctly', 'communicate', 'issue', 'offer', 'rate', 'deduction', 'autopay', 'showing', 'account', 'told', 'go', 'application', 'anyway', 'account', 'opened', 'could', 'add', 'autopay', 'receive', 'discount', 'way', 'since', 'account', 'opened', 'called', 'call', 'center', 'least', 'time', 'trying', 'receive', 'autopay_discount', 'first', 'time', 'told', 'going', 'applied', 'still', 'seen', 'additionally', 'last', 'time', 'called', 'week_ago', 'asked_speak_manager', 'told', 'would', 'take', 'day', 'get', 'back', 'still', 'yet_hear', 'back', 'business_day', 'later', 'told', 'would', 'receiving', 'autopay_discount', 'receiving', 'opened', 'account', 'company', 'lying', 'rate', 'loan', 'going', 'receive', 'dont', 'autopay', 'initiate', 'autopays', 'go', 'far', 'additional', 'issue', 'told', 'rate', 'going', 'based', 'month', 'libor_rate', 'published', 'wsj', 'month', 'none', 'rate', 'received', 'thus_far', 'match', 'rate', 'dont', 'really', 'know', 'tried_

In [ ]:
# As we can see above, out attempt at lemmatizing the data didn't work. We will just try it again.
# Here we are defining functions for bigrams, trigrams, and lemmatizing the data

def make_bigrams(data):
    return [bigram_mod[doc] for doc in data]

def make_trigrams(data):
    return [trigram_mod[bigram_mod[doc]] for doc in data]

def lemmatization(data, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    complaints_out = []
    for complaint in data:
        doc = nlp(" ".join(complaint))
        complaints_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return complaints_out

In [ ]:
!python3 -m spacy download en
# Now we call the functions we build above
data_words_bigrams = make_bigrams(data)
nlp = spacy.load("en", disable=["parser", "ner"])
data_lemmatized = lemmatization(data_words_bigrams)

print(data_lemmatized[:1])

     |████████████████████████████████| 12.0 MB 57.9 MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047106 sha256=ab1e5c7a1fdee7daf1dd990d09f32565ff6df9484ca2661fd7bfe8e0cbeeedee
  Stored in directory: /tmp/pip-ephem-wheel-cache-34_v_uf4/wheels/b7/0d/f0/7ecae8427c515065d75410989e15e5785dd3975fe06e795cd9
Successfully built en-core-web-sm
You should consider upgrading via the '/opt/venv/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/opt/venv/lib/python3.7/site-packages/en_core_web_sm -->
/opt/venv/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')
[['apply', 'loan', 'account', 'correctly', 'communicate', 'issue', 'offer', 'rate', 'deduction', 'autopay', 'show', 'account', 'tell', 'go', 'application', 'account', 'open', 'could', 'add', 'autopay', 'receive', 'discount', 'way', 'ac

At this point we've created some bi_grams and tri_grams. Now we need to create a dictionary and corpus that's needed for topic modeling.

In [ ]:
# Create dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
complaints = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(complaint) for complaint in complaints]

print("Corpus format: (word_id, work_frequency)")
print(corpus[:1])
print()
print("Readable version of term-frequency:")
print([[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]])

Corpus format: (word_id, work_frequency)
[[(0, 5), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 4), (8, 2), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 5), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 2), (33, 1), (34, 1), (35, 2), (36, 1), (37, 1), (38, 3), (39, 1), (40, 1), (41, 1), (42, 5), (43, 1), (44, 6), (45, 1), (46, 2), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 5), (53, 1), (54, 5), (55, 1), (56, 1), (57, 1), (58, 2), (59, 1)]]

Readable version of term-frequency:
[[('account', 5), ('add', 1), ('additional', 1), ('additionally', 1), ('application', 1), ('apply', 2), ('ask', 1), ('autopay', 4), ('back', 2), ('base', 1), ('call', 2), ('center', 1), ('communicate', 1), ('company', 1), ('correctly', 1), ('could', 1), ('day', 1), ('deduction', 1), ('discount', 1), ('do', 1), ('far', 1), ('first', 1), ('get', 1), ('go', 5), ('hear', 1), ('helpful', 1),

Now that we've prepared the data we are going to create and use a new function to test the other hypertunning parameters for an lda model, hopefully find one that's more accurate.

In [ ]:
# Function to create the model and calculate the coherence score
def compute_coherence_values(corpus, dictionary, a, b):
        lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                              id2word=id2word,
                                              num_topics=8,
                                              random_state=100,
                                              chunksize=100,
                                              passes=10,
                                              alpha=a,
                                              eta=b,
                                              per_word_topics=True)
        coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence="c_v")
        return coherence_model_lda.get_coherence()

From the previous notebook we decided to go with 8 being the number of topics, but the graph we previously looked at just showed a peak at 8 and then decreasing after that (we didn't even try 10, well at that moment we didn't it was tried later and showed it was a bit lower than 8, but to get the exact number of best topics we will just try each number from 2 up to 10).

This next part is going to take a long time, but should be worth it to get the most optimal hyperparameters for this model and dataset.

In [ ]:
import numpy as np
import tqdm

grid = {}
grid["Validation_Set"] = {}

# Alpha parameter
alpha = list(np.arange(0.01, 1, -0.3))
alpha.append("symmetric")
alpha.append("asymmetric")

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append("symmetric")

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.25)),
                # gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.5)),
                gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
                corpus]
corpus_title = ["75% Corpus", "100% Corpus"]
model_results = {"Validation_Set": [],
                "Alpha": [],
                "Beta": [],
                "Coherence": []}
    
# iterate through validation corpuses
for i in range(len(corpus_sets)):
    # iterate through alpha values
    for a in alpha: 
        #iterate though beta values
        for b in beta:
            # get the coherence score for the given parameters
            cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, a=a, b=b)
            # save the model result

            model_results["Validation_Set"].append(corpus_title[i])
            model_results["Alpha"].append(a)
            model_results["Beta"].append(b)
            model_results["Coherence"].append(cv)

pd.DataFrame(model_results).to_csv("../lda_tuning_results.csv", index=False)

The best coherence score we were able to get was in this instace:

    Validation_set, Alpha, Beta, Coherence
    100% Corpus,symmetric,0.9099999999999999,0.42464173686329043

Which is interesting, because before this testing with our previous notebook we attained a coherence score of 0.44, 
which is just shy of it. I think the part that made a difference was in preprocessing our data. For this notebook
the bigram and trigram models were slightly changed and common words were not removed. I'm going to work a litle more on 
optimizing the model by trying to get a mallet LDA to work on deepnote (which it wouldn't work on my computer for 
some odd reason). After optimizing the model, I will go back and try tweaking the preprocessing function and the
hyperparameters for the bigram and trigram models. 

Update: The Mallet LDA model doesn't work on this environment either. After spending sometime researching the error, we will just 
use the normal model. 

In [ ]:
# Create the optimal model
optimal_lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                              id2word=id2word,
                                              num_topics=8,
                                              random_state=100,
                                              chunksize=100,
                                              passes=10,
                                              alpha="symmetric",
                                              eta=0.909999999,
                                              per_word_topics=True)

In [ ]:
# Compute coherence score
coherence_model_lda = CoherenceModel(model=optimal_lda_model, texts=data_lemmatized, dictionary=id2word, coherence="c_v")
coherence_lda = coherence_model_lda.get_coherence()
print("\nCoherence score:", coherence_lda)


Coherence score: 0.44993563970976114


Now we've got a coherence score of 0.45, which is definitely the best we've gotten so far. Now that we've found the optimal model,
we can go ahead and work on the preprocessing a bit more and make the data coming in to the model more "optimal" to get "the most
optimal model".